In [1]:
!hdfs dfs -ls /labs/slaba02/DO_record_per_line.json

-rw-r--r--   3 hdfs hdfs   69519728 2022-01-06 18:46 /labs/slaba02/DO_record_per_line.json


In [2]:
!/share/submission-files/slaba02/lab02.json

/bin/sh: 1: /share/submission-files/slaba02/lab02.json: not found


In [3]:
import os
import sys
import json 
import re

In [4]:
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [5]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
from  pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import udf
from pyspark.ml.feature import Normalizer
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

spark.conf.set( "spark.sql.crossJoin.enabled" , "true" )

In [6]:
data = sc.textFile("/labs/slaba02/DO_record_per_line.json")

In [7]:
data.getNumPartitions()

2

In [8]:
# data.take(1)[0]

In [9]:
data_ = data.map(lambda x: json.loads(x))

In [10]:
# data_.count()

In [11]:
schema = StructType([
    StructField('lang', StringType(), True),
    StructField('name', StringType(), True),
    StructField('cat', StringType(), True),
    StructField('provider', StringType(), True),
    StructField('id', IntegerType(), True),
    StructField('desc', StringType(), True),
])

df = spark.createDataFrame(data_, schema=schema)

In [12]:
df.count()

28153

In [13]:
# df.show(2, vertical=True)

In [14]:
# df.where("id=16704").show()

In [15]:
courses = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], 
           [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], 
           [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], 
           [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], 
           [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'],
           [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

In [16]:
regex = re.compile(u'[\w\d]{2,}', re.U)
my_udf = udf(lambda x: regex.findall(x.lower()), ArrayType(StringType()))

df = df.withColumn('list', my_udf(F.col('desc')))

In [17]:
hashingTF = HashingTF(numFeatures=10000, inputCol='list', outputCol='tf')
tf = hashingTF.transform(df)

idf = IDF(inputCol="tf", outputCol="feature").fit(tf)
tfidf = idf.transform(tf).cache()

normalizer = Normalizer(inputCol="feature", outputCol="norm")
data = normalizer.transform(tfidf).cache()

In [18]:
dot_udf = udf(lambda x,y: float(x.dot(y)), DoubleType())
dict_ = dict({})

for course in courses:    
    curr_course_df = data.filter(F.col('id') == course[0]).alias('curr_course_df')
    bag_of_cousres = data.filter(F.col('lang') == course[1]).alias('bag_of_cousres')
    
    curr_top = curr_course_df.join(bag_of_cousres, curr_course_df.id != bag_of_cousres.id)\
            .select(
                F.col('bag_of_cousres.id').alias('id'),
                F.col('bag_of_cousres.name').alias('name'),
                dot_udf('curr_course_df.norm', 'bag_of_cousres.norm').alias('dot')
                )\
            .sort(['dot', 'name', 'id'], ascending=[False, True, True])\
            .limit(10)\
            .collect()
    
    curr_list = [int(row.id) for row in curr_top]
    dict_[str(course[0])] = curr_list

In [19]:
with open("/data/home/an.en/lab02.json", "w") as outfile:
    json.dump(dict_, outfile)

In [20]:
spark.stop()